In [ ]:
%pylab inline
import os, glob, pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
pylab.rcParams['figure.figsize'] = (10, 6)

In [ ]:
import sys
sys.path.append("../")
import muscle_utils

In [ ]:
import parest
ts = 0.01
padding = int(10/ts)

In [ ]:
FILE_MSE = {}

for file in glob.glob("/afs/ies.auc.dk/group/17gr1035/Private/exo-arm/software/model/muscles/parameter_estimation/*.pickle"):
    with open(file, 'rb') as tmp:
        values = pickle.load(tmp)
        FILE_MSE[file] = values['MSE']

best_parest = min(FILE_MSE, key=FILE_MSE.get)

pars = {}
with open(best_parest, 'rb') as tmp:
    pars = pickle.load(tmp)

triceps_pars = [
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['C1'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['C2'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['A'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['d'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['max_length'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['optimal_fiber_length'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['tensor_slack_length'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['max_force'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['alpha'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['Spe'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['Sse'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['phi_m'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['phi_v'],
]

biceps_pars = [
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['C1'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['C2'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['A'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['d'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['max_length'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['optimal_fiber_length'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['tensor_slack_length'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['max_force'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['alpha'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['Spe'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['Sse'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['phi_m'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['phi_v'],
]

brachialis_pars = [
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['C1'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['C2'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['A'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['d'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['max_length'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['optimal_fiber_length'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['tensor_slack_length'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['max_force'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['alpha'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['Spe'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['Sse'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['phi_m'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIALIS]['phi_v'],
]

brachioradialis_pars = [
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['C1'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['C2'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['A'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['d'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['max_length'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['optimal_fiber_length'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['tensor_slack_length'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['max_force'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['alpha'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['Spe'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['Sse'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['phi_m'],
    pars[muscle_utils.MUSCLE_NAME.BRACHIORADIALIS]['phi_v'],
]



params = triceps_pars + biceps_pars + brachialis_pars + brachioradialis_pars

In [ ]:
torque = []
for tau_tmp in parest.torque_out:
    torque = torque + list(tau_tmp)

In [ ]:
estimated = parest.FlexProblem()
muscles = estimated.set_up_muscles(params)
tau, act, m_tau = estimated.simulate(muscles)
act = np.array(act)

In [ ]:
plt.plot(tau)
plt.plot(torque)

In [ ]:
# a little filter
nyq = 0.5 * 100
cut = 1
low = cut / nyq
b, a = signal.butter(4, low, 'low')

In [ ]:
start = len(parest.mre7.emg0_in) + padding
stop = start + len(parest.mre8.emg0_in)
ext_est = tau[start:stop]
ext = torque[start:stop]
ext_act = act[:,start:stop]
ext_time = np.linspace(0, (stop-start)*0.01, stop-start)

ext_emg = np.concatenate((parest.mre8.emg0_in[0:], parest.mre8.emg1_in[0:]), 1).transpose()

plt.plot(ext_time, ext)
plt.plot(ext_time, ext_est)
plt.plot(ext_time, signal.lfilter(b, a, ext_est))

In [ ]:
start = \
len(parest.mre7.emg0_in) + padding + \
len(parest.mre8.emg0_in) + padding + \
len(parest.mmre1.emg0_in) + padding

stop = start + len(parest.mrf1.emg0_in)
flex_est = tau[start:stop]
flex = torque[start:stop]
flex_act = act[:,start:stop]
flex_time = np.linspace(0, (stop-start)*0.01, stop-start)

flex_emg = np.concatenate((parest.mrf1.emg0_in[0:],parest.mrf1.emg1_in[0:]), 1).transpose()

plt.plot(flex_time, flex)
plt.plot(flex_time, flex_est)
plt.plot(flex_time, signal.lfilter(b, a, flex_est))

In [ ]:
HEADER = \
    'TIME, MEAS, EST, EST_LP,' + \
    'TRICEPS, BICEPS, BRACHIALIS, BRACHIORADIALIS,' + \
    'POD1, POD2, POD3, POD4, POD5, POD6, POD7, POD8,' + \
    'POD1, POD2, POD3, POD4, POD5, POD6, POD7, POD8'

In [ ]:
np.savetxt(
    'est_ext_4m.dat', 
    np.vstack(
        [
            ext_time, 
            np.array(ext),
            np.array(ext_est), 
            signal.lfilter(b, a, ext_est),
            ext_act,
            ext_emg
        ]
    ).transpose(), 
    header=HEADER,
    comments=''
)

In [ ]:
np.savetxt(
    'est_flex_4m.dat', 
    np.vstack(
        [
            flex_time, 
            np.array(flex),
            np.array(flex_est), 
            signal.lfilter(b, a, flex_est),
            flex_act,
            flex_emg
        ]
    ).transpose(), 
    header=HEADER, 
    comments=''
)

In [ ]:
est = np.array(tau)
meas = np.array(torque)
mse = sum( (est - meas )**2 ) / len(meas)
print('MSE:', mse)